In [109]:
# For Matrix Operations
import sklearn
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
import numpy as np
import pandas as pd

# For Plotting
from matplotlib import pyplot as plt
import plotly
import plotly.graph_objects as go
import mplcursors

# for Geospatial
import geopandas as gpd
from gerrychain import Graph
import networkx as nx

# For TDA
import gudhi
import gudhi.wasserstein
import gudhi.hera
import ot

# VERSIONS USED
# !pip install gerrychain==0.2.22
# !pip install shapely==2.0.2
# !pip install geopandas==0.12.2
#################

gdf = gpd.read_file('TDA-Class-Notebook4/NC_VTD/NC_VTD.shp')
Elections = [
            # 2008 Gubernatorial election (democrat won)
            ('EL08G_GV_D', 'EL08G_GV_R'), 
            # 2008 Senate election (democrat won)
            ( 'EL08G_USS_', 'EL08G_US_1'),
            # 2010 Senate election (republican won)
            ( 'EL10G_USS_', 'EL10G_US_1'),
            # 2012 Gubernatorial election (republican won) 
            ( 'EL12G_GV_D', 'EL12G_GV_R'),
            # 2014 Senate election (republican won)
            ( 'EL14G_US_1', 'EL14G_USS_'),
            # 2012 Presidential election
            ( 'EL12G_PR_D', 'EL12G_PR_R'),
            # 2016 Presidential election
            ( 'EL16G_PR_D', 'EL16G_PR_R'),
            # 2016 Senate election (republican won)
            ( 'EL16G_US_1', 'EL16G_USS_'),
            # 2016 Gubernatorial election (democrat won)
            ('EL16G_GV_D', 'EL16G_GV_R')]

Election_Names = ['2008 Gubernatorial election', 
                  '2008 Senate election', 
                  '2010 Senate election', 
                  '2012 Gubernatorial election', 
                  '2014 Senate election', 
                  '2012 Presidential election', 
                  '2016 Presidential election', 
                  '2016 Senate election', 
                  '2016 Gubernatorial election']

# Edited to have one less red
Winners = ['blue'] * 2 + ['red'] * 6 + ['blue']
graph = Graph.from_geodataframe(gdf, adjacency='queen', ignore_errors = True)


In [157]:
def Adjacency_Persistence(graph, Hs_D, Hs_R, dimension = None):
    for t in range(len(Hs_D)):
        scomplex = gudhi.SimplexTree()
        for i in graph.nodes:
            scomplex.insert([i]) #add a 0-simplex, given as a list with just one vertex

        for e in graph.edges:
            scomplex.insert([e[0], e[1]]) #add a 1-simplex

    # Reassigning each vertex value
        for v in graph.nodes:
            graph.nodes[v]['Democrat Homotopy'] = Hs_D[t][v]
            graph.nodes[v]['Republican Homotopy'] = Hs_R[t][v]
    # Filtering
        for v in graph.nodes:
            scomplex.assign_filtration(
            [v], #we have to put [] here because a 0-simplex is technically a list with one element.
            filtration=graph.nodes[v]['Democrat Homotopy']/(graph.nodes[v]['Republican Homotopy']+graph.nodes[v]['Democrat Homotopy'])
            )
    
        scomplex.make_filtration_non_decreasing()   

        Filtration = [v for v in scomplex.get_skeleton(0)]
        if dimension == None:  
            persistence = scomplex.persistence()

        else:
            scomplex.compute_persistence()
            persistence = scomplex.persistence_intervals_in_dimension(dimension)
        
    return persistence


In [154]:

PDDS_0 = [Adjacency_Persistence(graph, [gdf[Election[0]]], [gdf[Election[1]]], 0) for Election in Elections]
for x, PDD in enumerate(PDDS_0):
    gudhi.plot_persistence_diagram(PDD)
    plt.title('{}'.format(Election_Names[x]))
    plt.show()


start
start
start
start
start
start
start
start
